## New Drag Regression Model ##

This script runs a pipeline to load data, format data, train the neural network, evaluate it's accuracy, and save the model.

This drag takes into account the specfic speed, draft, and length of the vessel in its prediction




In [1]:
#import tools

import csv
import numpy as np
import json
import math
import matplotlib.pyplot as plt
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
'''
import multiprocessing as mp

from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score

import sklearn.preprocessing as PP


import sys

sys.path.append('/home/ada/Documents/HullParameterization')

from HullParameterization import Hull_Parameterization as HP

import ModifiedMichellCw as MCW

rho = 1025 #(kg/m^3) saltwater density  

"""
Modified Michell CW calcs to condsider:

MCW.CalcDrag(U,LOA,WL,[CW], T, SA, rho=1025)


MCW.Calc_Cf(U,WL)

MCW.interp_CW(Fn=Fn,T=T,CW=CW)

"""

'\nModified Michell CW calcs to condsider:\n\nMCW.CalcDrag(U,LOA,WL,[CW], T, SA, rho=1025)\n\n\nMCW.Calc_Cf(U,WL)\n\nMCW.interp_CW(Fn=Fn,T=T,CW=CW)\n\n'

In [2]:
#CPU Multithreading

def run_MAP_multiprocessing(func, argument_list, chunksize = None, show_prog = True):
    """Run function in parallel
    Parameters
    ----------
    func:          function
                    Python function to run in parallel.
    argument_list: list [N]
                    List of arguments to be passed to the function in each parallel run.
            
    show_prog:     boolean
                    If true a progress bas will be displayed to show progress. Default: True.
    Returns
    -------
    output:        list [N,]
                    outputs of the function for the given arguments.
    """
    #Reserve 2 threads for other Tasks
    #pool = mp.Pool(processes=mp.cpu_count()-2)
    
    if show_prog:            
        result_list_tqdm = []
        for result in tqdm(pool.map(func=func, iterable=argument_list,chunksize=chunksize), total=len(argument_list),position=0, leave=True):
            result_list_tqdm.append(result)
    else:
        result_list_tqdm = []
        for result in pool.map(func=func, iterable=argument_list,chunksize=chunksize):
            result_list_tqdm.append(result)

    return result_list_tqdm


def run_IMAP_multiprocessing(func, argument_list, chunksize = None, show_prog = True):
    """Run function in parallel
    Parameters
    ----------
    func:          function
                    Python function to run in parallel.
    argument_list: list [N]
                    List of arguments to be passed to the function in each parallel run.
            
    show_prog:     boolean
                    If true a progress bas will be displayed to show progress. Default: True.
    Returns
    -------
    output:        list [N,]
                    outputs of the function for the given arguments.
    """
    #Reserve 2 threads for other Tasks
    #pool = mp.Pool(processes=mp.cpu_count()-2)
    
    if show_prog:            
        result_list_tqdm = []
        for result in tqdm(pool.imap(func=func, iterable=argument_list,chunksize=chunksize), total=len(argument_list),position=0, leave=True):
            result_list_tqdm.append(result)
    else:
        result_list_tqdm = []
        for result in pool.imap(func=func, iterable=argument_list,chunksize=chunksize):
            result_list_tqdm.append(result)

    return result_list_tqdm




## Load and Format the Training Data ##

In [3]:
#Load XLimits

X_LIMITS = np.load('./data/X_LIMITS.npy')

print(X_LIMITS.shape)

#Load in Volume Prediction for Now
DesVecName = 'Input_Vectors.csv'
YName = 'GeometricMeasures/Volume.csv'
CwName = 'GeometricMeasures/Cw.csv'
DS_path = '/home/ada/Documents//Hull_DataSet/'

folder_roots = ['Constrained_Randomized_Set_1', 
                'Constrained_Randomized_Set_2',
                'Constrained_Randomized_Set_3',
                'Diffusion_Aug_Set_1',
                'Diffusion_Aug_Set_2']
DesVec = []
Cw = []

for i in range(0,len(folder_roots)):
    path = DS_path + folder_roots[i] + '/'    
    #Location of Design Vectors
    with open(path + DesVecName) as csvfile:
        reader = csv.reader(csvfile)
        for count, row in enumerate(reader):
            if count != 0:
                DesVec.append(row)

    #Location of Cw Vectors
    with open(path + CwName) as csvfile:
        reader = csv.reader(csvfile)
        for count, row in enumerate(reader):
            if count != 0:
                Cw.append(row)
'''
    #Location of Vol Vectors
    with open(path + YName) as csvfile:
        reader = csv.reader(csvfile)
        for count, row in enumerate(reader):
            if count != 0:
                YVec.append(row)
            else:
                labels = np.array(row)
'''
DesVec = np.array(DesVec)
DesVec = DesVec.astype(np.float32())

np.save('DesVec_82k.npy',DesVec)

Cw = np.array(Cw)
Cw = Cw.astype(np.float32())

np.save('Cw_82k.npy',Cw)

#YVec = np.array(YVec)
#YVec = YVec.astype(np.float32())

#Normalize Volume to LogScale
#Y_log = np.log10(YVec)

def Performance_Metric(X):
    hull = HP(X)
    Z = hull.Calc_VolumeProperties(101,1000)
    
    return np.divide(hull.Volumes,X[0]**3.0)

def Calc_GeometricProperties(x):
    '''
    This function takes in a Ship Design Vector and calculates the volumetric properties of the hull 
    
    It returns the values for:
    
    Z / L             -> nondimensialized vector for the height at which each value was measured
    Volume / L^3
    Area of Waterplane / L^2
    Longitudinal Centers of Buoyancy/L
    Vertical Center of Buoyancy / L
    Longitudinal Center of Flotation / L
    Ixx / L^4
    Iyy / L^4
    
    where L = LOA of the design vector ( x[0])
    
    This function is written to be paralellized   
    
    '''
    
    hull = HP(x)
       
    Z = hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)
    
    L = x[0]
    
    z = np.divide(Z,L)
    Vol = np.divide(hull.Volumes,L**3.0)
    WP = np.divide(hull.Areas_WP,L**2.0)
    LCF = np.divide(hull.LCFs,L)
    Ixx = np.divide(hull.I_WP[:,0],L**4.0)
    Iyy = np.divide(hull.I_WP[:,1],L**4.0)
    LCB = np.divide(hull.VolumeCentroids[:,0],L)
    VCB = np.divide(hull.VolumeCentroids[:,0],L)
    WSA = np.divide(hull.Area_WS,L**2.0)
    WL = np.divide(hull.WL_Lengths,L)
    
    
    return np.concatenate((z,Vol,WP,LCB,VCB,LCF,Ixx,Iyy,WSA,WL),axis=0)

(44, 2)


In [4]:
# Now lets clean up X

idx_BBFactors = [33,34,35,36,37]
idx_BB = 31

idx_SBFactors = [38,39,40,41,42,43,44]
idx_SB = 32

for i in range(0,len(DesVec)):
    
    DesVec[i,idx_BBFactors] = DesVec[i,idx_BB] * DesVec[i,idx_BBFactors] 
    DesVec[i,idx_SBFactors] = DesVec[i,idx_SB] * DesVec[i,idx_SBFactors]



In [5]:
'''

#Compute Length Ratios:

#Run multiprocessing to calculate the length ratios
def Calc_LengthRatios(x):
    hull = HP(x)
    Loa_wBulb = hull.Calc_LOA_wBulb()/x[0]
    Beam_mid = hull.Calc_Max_Beam_midship()/x[0]
    hull.PCMeasurement = hull.gen_pointCloud(NUM_WL = 101, PointsPerWL = 1000, bit_GridOrList = 1, Z =  np.linspace(0.00001,hull.Dd, num=101))
    Beam_pc   = hull.Calc_Max_Beam_PC()/x[0]
    return [Loa_wBulb,Beam_mid,Beam_pc]

CHUNKS = 128
print('Calculating Length Ratios...')
print('Threads: ' + str(mp.cpu_count()))
pool = mp.Pool(processes=mp.cpu_count()-2)
Len_Ratios = run_IMAP_multiprocessing(Calc_LengthRatios, DesVec,chunksize=CHUNKS,show_prog=True)
Len_Ratios = np.array(Len_Ratios)
np.save('Length_Ratios.npy',Len_Ratios)
print('Length Ratios Calculations Complete!')

'''


Y = np.load('GeometricMeasures.npy')
print(Y.shape)



Calculating Length Ratios...
Threads: 32


100%|██████████| 82168/82168 [05:39<00:00, 242.20it/s]

Length Ratios Calculations Complete!
(82168, 1010)


### Quantile Normalize the Design Vectors: ###

In [129]:
class Data_Normalizer:
    def __init__(self, X_LL_Scaled, X_UL_Scaled,datalength):
        
        self.normalizer = PP.QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(datalength // 30, 1000), 10),
            subsample=int(1e9)
            )
        
        self.X_LL_Scaled = X_LL_Scaled
        self.X_UL_Scaled = X_UL_Scaled
        
        self.X_LL_norm = np.zeros((1,len(X_LL_Scaled)))
        self.X_UL_norm = np.zeros((1,len(X_LL_Scaled)))
        
        self.X_mean = np.zeros((1,len(X_LL_Scaled)))
        self.X_std = np.zeros((1,len(X_LL_Scaled)))
        
    def fit_Data(self,X):
        
        
        
        x = 2.0*(X-self.X_LL_Scaled)/(self.X_UL_Scaled- self.X_LL_Scaled) - 1.0
        
        self.normalizer.fit(x)
        x = self.normalizer.transform(x) # Scale Dataset between 
        #x = (X-self.X_LL_Scaled)/(self.X_UL_Scaled- self.X_LL_Scaled)
        

        return x
    
    def transform_Data(self,X):
        x = 2.0*(X-self.X_LL_Scaled)/(self.X_UL_Scaled- self.X_LL_Scaled) - 1.0
        
        
        x = self.normalizer.transform(x)
        return x
        

    def scale_X(self,z):
        #rescales data
        z = self.normalizer.inverse_transform(z)
        scaled = (z + 1.0) * 0.5 * (self.X_UL_Scaled - self.X_LL_Scaled) + self.X_LL_Scaled
        #scaled = z* (self.X_UL_Scaled - self.X_LL_Scaled) + self.X_LL_Scaled

        '''
        x = self.normalizer.inverse_transform(x)
        
        #scaled = x* (self.X_UL_norm - self.X_LL_norm) + self.X_LL_norm
        '''
        #z = (z + 1.0) * 0.5 * (8.0) + 4.0
       
        #scaled = z*self.X_std + self.X_mean
        #scaled = self.normalizer.inverse_transform(scaled)
        return scaled

## Build Regression Model Class ##



In [130]:
import torch
import torch.nn as nn
import torch.nn.functional as F  



class Drag_Regression_ResNet(torch.nn.Module):
    def __init__(self, Reg_Dict):
        nn.Module.__init__(self)
        
        self.xdim = Reg_Dict['xdim']+3 # Add 3 Draft, Velocity (Froude Number), and Length scale (LOA)
        self.ydim = 1
        self.tdim = Reg_Dict['tdim']
        self.net = Reg_Dict['net']
        
        self.fc = nn.ModuleList()
        
        self.fc.append(self.LinLayer(self.tdim,self.net[0]))
        
        for i in range(1, len(self.net)):
            self.fc.append(self.LinLayer(self.net[i-1],self.net[i]))
            
        self.fc.append(self.LinLayer(self.net[-1], self.tdim))
        '''
        #self.tc = nn.ModuleList()

        #for i in range(0, len(self.net)):
            self.tc.append(self.LinLayer(self.tdim,self.net[i]))
        self.tc.append(self.LinLayer(self.tdim, self.tdim))
        '''
        self.finalLayer = nn.Sequential(nn.Linear(self.tdim, self.ydim))
        
    
        self.X_embed = nn.Linear(self.xdim, self.tdim)
        #self.T_embed = nn.Linear(self.ydim, self.tdim)
       
        
    def LinLayer(self, dimi, dimo):
        
        return nn.Sequential(nn.Linear(dimi,dimo),
                             nn.SiLU(),
                             nn.LayerNorm(dimo),
                             nn.Dropout(p=0.1))
    
    def forward(self, x):
        x = self.X_embed(x)
    
        res_x = x

        for i in range(0,len(self.fc)):
            x = self.fc[i](x)
        
        x = torch.add(x,res_x)
        x = self.finalLayer(x)
        
        return x


## Make Training Environment ##

In [131]:
class Drag_Regressor_Training_Env:
    def __init__(self, Drag_Reg_Dict, DesVec, CwVec, WL, WSA, Fn_range = [0.1,0.45], T_range = [.25,.67], LOA_range = [3,450]):

        self.Reg_Dict = Drag_Reg_Dict
        self.DesVec = DesVec

        self.QT = Data_Normalizer(X_LIMITS[:,0],X_LIMITS[:,1],len(DesVec))
        
        self.X = np.copy(DesVec[:,1:])

        # Quantile Transform X:
        self.X = self.QT.fit_Data(self.X)

        self.CwVec = CwVec
        self.WL = WL
        self.WSA = WSA  
        self.Fn_range = Fn_range   
        self.T_range = T_range
        self.LOA_range = LOA_range
        
        self.num_WL_Steps = 101
        self.T_vec = np.linspace(0,1,self.num_WL_Steps)
        
      
        self.model = Drag_Regression_ResNet(self.Reg_Dict)
        self.device =torch.device(self.Reg_Dict['device_name'])

        self.model.to(self.device)

        self.data_length = len(self.X)
        self.batch_size = self.Reg_Dict['batch_size']
        self.num_epochs = self.Reg_Dict['Training_Epochs']
        
        lr = self.Reg_Dict['lr']
        self.init_lr = lr
        weight_decay = self.Reg_Dict['weight_decay']
        
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=lr, weight_decay=weight_decay)


    '''
    ==============================================================================
    Base Regression Training Functions
    ==============================================================================
    '''
    def batch_CT(self,x, A, batch_size = None):
        '''
        This function takes in a batch of design vectors and outputs the corresponding CT values
        '''
        if batch_size == None:
            batch_size = self.batch_size
        
         #Random Log Scale LOA 
        log10_LOA = np.random.uniform(np.log10(self.LOA_range[0]), np.log10(self.LOA_range[1]),(batch_size,))
        LOA = 10**log10_LOA       

        #Random Waterline

        t = np.random.uniform(self.T_range[0], self.T_range[1], (batch_size,))
        t_tens = torch.tensor(t[:,np.newaxis]).float().to(self.device)

        #Random Froude Number
        Fn = np.random.uniform(self.Fn_range[0],self.Fn_range[1],(batch_size,))

        WL = np.array([HP.interp(self.WL[A[i]],self.T_vec,t[i]) for i in range(0,len(t))]) #Non-dimensionalized Waterline Length    

        U = Fn * np.sqrt(9.81*LOA*WL)

        WSA = np.array([HP.interp(self.WSA[A[i]],self.T_vec,t[i]) for i in range(0,len(t))]) #Non-dimensionalized Wetted Surface Area

        #Interpolate Cw and calc Cf
        CW = np.zeros((batch_size,))
        CF = np.zeros((batch_size,))   

        for i in range(0,batch_size):
            CW[i] = MCW.interp_CW(Fn[i], t[i], self.CwVec[A[i]])
            CF[i] = MCW.Calc_Cf(U[i],LOA[i]*WL[i])*WSA[i] #CF = Rf/(0.5*rho*U^2*WSA) * WSA/LOA**2 = Cf * WSA/LOA**2
        
        #Put CT on a log scale and Batch
        CT = np.log10(CW + CF)
        
        y_batch = torch.tensor(CT[:,np.newaxis]).float().to(self.device)

        #Batch x
        log10_LOA = torch.tensor(log10_LOA[:,np.newaxis]).float().to(self.device)
        Fn = torch.tensor(Fn[:,np.newaxis]).float().to(self.device)
        
        x_batch = torch.cat((x,t_tens,Fn,log10_LOA),dim=1)
        
        return x_batch,y_batch
    
    def run_regressor_step(self,x,y):
        self.optimizer.zero_grad()

        ones = torch.ones_like(y)

        predicted_y = self.model(x)
        
        loss =  F.mse_loss(predicted_y, y)
        #print(loss)
        
        loss.backward()
        self.optimizer.step()
        
        return loss  
    
    def run_train_regressors_loop(self,batches_per_epoch=64, subsample_per_batch = 64):
            
            num_batches = self.data_length // self.batch_size
        
            batches_per_epoch = min(num_batches,batches_per_epoch)

            
            
            print('Regressor Model Training...')

            for i in tqdm(range(0,self.num_epochs)):

                for j in range(0,batches_per_epoch):
                    
                    A = np.random.randint(0,self.data_length,self.batch_size)
                    x = torch.tensor(self.X[A]).float().to(self.device) 

                
                    for k in range(0,subsample_per_batch):
                        x_batch,y_batch = self.batch_CT(x,A)
                       
                        loss = self.run_regressor_step(x_batch,y_batch)
                if i % 1000 == 0:

                    print('Epoch: ' + str(i) + ' Loss: ' + str(loss))   
                    
        
            print('Regression Model Training Complete!')

            self.model.eval()
            eval_size = 10000

            A = np.random.randint(0,self.data_length,eval_size)
            x_eval = torch.tensor(self.X[A]).float().to(self.device)

            x_eval,Y_calc = self.batch_CT(x_eval,A,eval_size)

            Y_calc = Y_calc.to(torch.device('cpu')).detach().numpy()

            Y_pred = self.model(x_eval)
            Y_pred = Y_pred.to(torch.device('cpu')).detach().numpy() 

            Rsq = r2_score(Y_calc, Y_pred)
            print("R2 score of Y:" + str(Rsq))


    # SAVE FUNCTIONS
        
    def load_trained_model(self):
        label = self.Reg_Dict['Model_Path']
        self.model.load_state_dict(torch.load(label))
        self.model.to(self.device)
        
    
    def Save_model(self,PATH):
        '''
        PATH is the path to the folder to store this in, including '/' at the end
       
        '''

        torch.save(self.model.state_dict(), PATH + self.Reg_Dict['Model_Label']+'.pth')
        
        JSON = json.dumps(self.Reg_Dict)
        f = open(PATH + self.Reg_Dict['Model_Label'] + '_Dict.json', 'w')
        f.write(JSON)
        f.close()

## Set Up Model Training ##

In [132]:
#Regression model Dict
nodes = 512

Reg_Dict = {
        'xdim' : len(DesVec[0])-1,              # Dimension of parametric design vector
        'ydim': 1,                              # trains regression model for each objective
        'tdim': nodes,                            # dimension of latent variable
        'net': [nodes,nodes,nodes,nodes],                       # network architecture        
        'Training_Epochs': 50000,  #30000             # number of training epochs
        'batch_size': 1024,                       # batch size
        'Model_Label': 'Regressor_CT',         # labels for regressors
                    
        'lr' : 0.001,                          # learning rate
        'weight_decay': 0.0,                   # weight decay
        'device_name': 'cuda:0'}    


num_WL_Steps = 101

WL = Y[:,9*num_WL_Steps:10*num_WL_Steps]
WSA = Y[:,8*num_WL_Steps:9*num_WL_Steps]

#Y_set = Y[:,8*num_WL_Steps:9*num_WL_Steps]
idx = np.where(np.isnan(WL))
#print(idx)

WL[idx] = 1.0 #fix nan to dummy value

idx = np.where(np.isnan(WSA))
#print(idx)

WSA[idx] = -5.0 #fix nan to dummy value


REG = Drag_Regressor_Training_Env(Reg_Dict, DesVec, Cw, WL, WSA)

model_size = 0
for p in REG.model.parameters():
        model_size += p.numel()

print('Model size: ', model_size)

REG.run_train_regressors_loop(batches_per_epoch=8, subsample_per_batch = 8)

REG.Save_model('./TrainedModels/')

Model size:  1343489
Regressor Model Training...


  0%|          | 1/50000 [00:00<7:46:06,  1.79it/s]

Epoch: 0 Loss: tensor(0.1898, device='cuda:0', grad_fn=<MseLossBackward0>)


  2%|▏         | 1001/50000 [09:15<7:31:31,  1.81it/s]

Epoch: 1000 Loss: tensor(0.0046, device='cuda:0', grad_fn=<MseLossBackward0>)


...

Epoch: 48000 Loss: tensor(0.0016, device='cuda:0', grad_fn=<MseLossBackward0>)


 98%|█████████▊| 49001/50000 [7:34:56<09:17,  1.79it/s]

Epoch: 49000 Loss: tensor(0.0018, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|██████████| 50000/50000 [7:44:12<00:00,  1.80it/s]

Regression Model Training Complete!
R2 score of Y:0.997389711775577


In [135]:
REG.model.eval()

sample_size = 100000


A = np.random.randint(0,REG.data_length,sample_size)
x_eval = torch.tensor(REG.X[A]).float().to(REG.device)

x_eval,Y_calc = REG.batch_CT(x_eval,A,sample_size)

Y_calc = Y_calc.to(torch.device('cpu')).detach().numpy()

Y_pred = REG.model(x_eval)
Y_pred = Y_pred.to(torch.device('cpu')).detach().numpy() 



#MAEP = np.mean(np.abs(np.power(10,Y_calc)-np.power(10,Y_pred)/np.power(10,Y_calc)))
MAEP = np.mean(np.abs(Y_calc-Y_pred)/np.abs(Y_calc))
print('Log scale MAEP: ' + str(MAEP*100.0) + '%')


Y_scaled_calc = 10**Y_calc
Y_scaled_pred = 10**Y_pred  

MAEP_scaled = np.mean(np.abs(Y_scaled_calc-Y_scaled_pred)/np.abs(Y_scaled_calc))
print('Scaled Ct MAEP: ' + str(MAEP_scaled*100.0) + '%')
Rsq = r2_score(Y_scaled_calc, Y_scaled_pred)

print("R2 score of Scaled Ct Prediction: " + str(Rsq))


# Calculate Rt

x_eval = x_eval.to(torch.device('cpu')).detach().numpy()

LOA = 10**x_eval[:,-1]
t = (x_eval[:,-3])

WL = np.array([HP.interp(REG.WL[A[i]],REG.T_vec,t[i]) for i in range(0,len(t))]) #Non-dimensionalized Waterline Length    


U = x_eval[:,-2]*np.sqrt(9.81*LOA*WL)

Rt_Calc = 0.5*rho*(U**2.0) * (LOA**2.0) * Y_scaled_calc[:,0]


Rt_Pred = 0.5*rho*(U**2.0) * (LOA**2.0) * Y_scaled_pred[:,0]



Rsq = r2_score(Y_scaled_calc, Y_scaled_pred)

print("R2 score of Scaled Rt Prediction: " + str(Rsq))
MAEP_scaled_Rt = np.mean(np.abs(Rt_Calc-Rt_Pred)/np.abs(Rt_Calc))
print('Scaled Rt MAEP: ' + str(MAEP_scaled_Rt*100.0) + '%')





Log scale MAEP: 1.4282303862273693%
Scaled Ct MAEP: 6.629853695631027%
R2 score of Scaled Ct Prediction: 0.9937638501995272
R2 score of Scaled Rt Prediction: 0.9937638501995272
Scaled Rt MAEP: 6.629852816703814%
